In [1]:
# -*- coding: utf-8 -*-
"""kindergartens_in_berlin_Data Extraction.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1kRxJhj3Yhdqq2xf_MLuk-3zqfCRdXA1y
"""

# data extraction
import os
import requests
import zipfile
import json
import pandas as pd
from io import BytesIO
import re
from google.colab import files

# Configuration
DATA_DIR = "/content/sources"
os.makedirs(os.path.join(DATA_DIR, "raw"), exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, "samples"), exist_ok=True)

def download_berlin_kitas():
    """Download official kindergarten data from Berlin Open Data Portal directly"""
    try:
        print("Downloading Berlin government data directly...")

        # Direct link to the ZIP file (update if changed in future)
        zip_url = "https://daten.berlin.de/sites/default/files/2023-03/kindertagesbetreuung_berlin.csv.zip"

        # Download and extract
        zip_response = requests.get(zip_url)
        zip_response.raise_for_status()

        with zipfile.ZipFile(BytesIO(zip_response.content)) as z:
            csv_name = next(f for f in z.namelist() if f.endswith('.csv'))
            with z.open(csv_name) as csv_file:
                content = csv_file.read().decode('utf-8-sig')
                clean_lines = [line for line in content.split('\n')
                               if line.strip() and not line.startswith((',,,,,', 'Anbieter'))]

                # Save raw full file
                raw_path = os.path.join(DATA_DIR, "raw", "berlin_kitas.csv")
                with open(raw_path, 'w', encoding='utf-8') as f:
                    f.write('\n'.join(clean_lines))

                # Save sample (first 100 lines)
                sample_path = os.path.join(DATA_DIR, "samples", "berlin_sample.csv")
                with open(sample_path, 'w', encoding='utf-8') as f:
                    f.write('\n'.join(clean_lines[:100]))

        print("Berlin data downloaded successfully")
        return True

    except Exception as e:
        print(f"Berlin data download failed: {str(e)}")
        return False

def download_osm_kitas():
    """Fetch kindergarten data from OpenStreetMap"""
    try:
        print("\nDownloading OSM data...")
        query = """
        [out:json];
        area[name="Berlin"]->.a;
        (node["amenity"="kindergarten"](area.a); way["amenity"="kindergarten"](area.a););
        out center;
        """
        response = requests.post(
            "https://overpass-api.de/api/interpreter",
            data={"data": query}
        )
        response.raise_for_status()
        data = response.json()

        # Save raw file
        raw_path = os.path.join(DATA_DIR, "raw", "osm_kitas.json")
        with open(raw_path, 'w') as f:
            json.dump(data, f)

        # Save sample (first 10 elements)
        sample_path = os.path.join(DATA_DIR, "samples", "osm_sample.json")
        with open(sample_path, 'w') as f:
            json.dump({"elements": data["elements"][:10]}, f)

        print("OSM data downloaded successfully")
        return True

    except Exception as e:
        print(f"OSM download failed: {str(e)}")
        return False

# Run downloads
print("=== Berlin Kindergartens Data Extraction ===")
berlin_success = download_berlin_kitas()
osm_success = download_osm_kitas()

if berlin_success or osm_success:
    # Create ZIP for download
    !zip -r /content/kita_data.zip /content/sources
    print("\nDownload the complete dataset:")
    files.download("/content/kita_data.zip")

    # Show samples
    if berlin_success:
        try:
            df = pd.read_csv(os.path.join(DATA_DIR, "samples", "berlin_sample.csv"), sep=';', nrows=5)
            print("\nBerlin sample data:")
            display(df)
        except Exception as e:
            print(f"Couldn't display sample: {str(e)}")

    if osm_success:
        try:
            with open(os.path.join(DATA_DIR, "samples", "osm_sample.json")) as f:
                print("\nOSM sample features:", len(json.load(f)["elements"]))
        except Exception as e:
            print(f"Couldn't load OSM sample: {str(e)}")
else:
    print("\nNo data downloaded. Check error messages above.")

"""**Step 2: Data Transformation**"""

# PR Branch: kindergartens-data-transformation
# transform_kindergartens.py
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import json
from shapely.geometry import Point
import os

# Configuration
DATA_DIR = "data"
os.makedirs(os.path.join(DATA_DIR, "processed"), exist_ok=True)

=== Berlin Kindergartens Data Extraction ===
Berlin data download failed: 404 Client Error: Not Found for url: https://daten.berlin.de/sites/default/files/2023-03/kindertagesbetreuung_berlin.csv.zip

OSM data downloaded successfully
  adding: content/sources/ (stored 0%)
  adding: content/sources/raw/ (stored 0%)
  adding: content/sources/raw/osm_kitas.json (deflated 79%)
  adding: content/sources/samples/ (stored 0%)
  adding: content/sources/samples/osm_sample.json (deflated 71%)

Download the complete dataset:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


OSM sample features: 10
